# 1. `tn_traveller_master_여행객 Master_E.csv` (가장 중심이 되는 파일)

이 파일은 여행객의 '정체성'을 담고 있으므로, 대부분의 정보를 활용합니다.

* **선택할 핵심 컬럼:**
    * `TRAVELER_ID`: **(필수)** 모든 데이터 병합의 기준이 되는 기본 키입니다.
    * **[인구통계]** `GENDER`, `AGE_GRP`, `MARR_STTS` (혼인상태), `FAMILY_MEMB` (가족 수), `JOB_NM` (직업), `INCOME` (개인소득), `HOUSE_INCOME` (가구소득)
        * **이유:** 페르소나를 정의하는 가장 기본적인 골격입니다. (예: 20대 미혼 여성, 40대 기혼 직장인 등)
    * **[여행 성향]** `TRAVEL_STYL_1` ~ `TRAVEL_STYL_8`
        * **이유:** 여행객의 선호도를 직접적으로 보여주는 매우 중요한 정보입니다. "어떤 스타일의 여행을 선호하는가?"를 나타냅니다.
    * **[여행 동기]** `TRAVEL_MOTIVE_1`, `TRAVEL_MOTIVE_2`, `TRAVEL_MOTIVE_3`
        * **이유:** "왜 여행을 떠나는가?"에 대한 정보로, 페르소나의 내적 동기를 파악하는 데 결정적입니다. (예: 휴식, 레저, 경험 등)
    * **[동반자 정보]** `TRAVEL_STATUS_ACCOMPANY` (동반 현황), `TRAVEL_COMPANIONS_NUM` (동반자 수)
        * **이유:** '나홀로', '커플', '자녀 동반' 등 페르소나를 구분하는 핵심적인 행동 특성입니다.

* **후순위 또는 제외 고려 컬럼:**
    * `RESIDENCE_SGG_CD` (거주지 시군구 코드): 너무 세분화되어 있어 군집화에 노이즈가 될 수 있습니다. '수도권/비수도권' 정도로 크게 묶어서 가공한다면 유용할 수 있습니다.
    * `EDU_NM`, `EDU_FNSH_SE` (학력 정보): 여행 스타일과의 직접적인 연관성이 다른 변수보다 낮을 수 있어 초기 분석에서는 제외를 고려할 수 있습니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

plt.rc('font', family='Malgun Gothic')  

# 음수 기호 깨짐 방지
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('./data/tn_traveller_master_여행객 Master_E.csv')

print(df.shape) # (2560, 36)
df.head()

(2560, 36)


,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,e004720,41,여,60,4,1.0,3,3,11,NaN,...,5,5,경기도,서울,2인 가족 여행,2023-07-16~2023-07-16,2,6.0,NaN,1
1,e000914,30,여,20,6,1.0,1,1,3,NaN,...,4,1,대전광역시,서울,나홀로 여행,2023-06-03~2023-06-03,1,7.0,10.0,0
2,e003564,41,여,30,7,1.0,2,4,2,NaN,...,1,7,경기도,경기,자녀 동반 여행,2023-06-24~2023-06-24,8,3.0,7.0,3
3,e000396,41,여,30,6,1.0,2,2,2,NaN,...,1,6,경기도,인천,2인 가족 여행,2023-05-20~2023-05-21,9,1.0,7.0,1
4,e001890,11,남,20,6,1.0,1,4,3,NaN,...,5,6,서울특별시,경기,2인 여행(가족 외),2023-06-04~2023-06-04,3,1.0,5.0,1


In [3]:
df.info()
# EDU_FNSH_SE(최종학력이수여부), JOB_ETC(직업_기타), HOUSE_INCOME(가구소득), TRAVEL_MOTIVE_3(세부여행동기)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   TRAVELER_ID                2560 non-null   object 
 1   RESIDENCE_SGG_CD           2560 non-null   int64  
 2   GENDER                     2560 non-null   object 
 3   AGE_GRP                    2560 non-null   int64  
 4   EDU_NM                     2560 non-null   int64  
 5   EDU_FNSH_SE                2558 non-null   float64
 6   MARR_STTS                  2560 non-null   int64  
 7   FAMILY_MEMB                2560 non-null   int64  
 8   JOB_NM                     2560 non-null   int64  
 9   JOB_ETC                    0 non-null      float64
 10  INCOME                     2560 non-null   int64  
 11  HOUSE_INCOME               2004 non-null   float64
 12  TRAVEL_TERM                2560 non-null   int64  
 13  TRAVEL_NUM                 2560 non-null   int64